# 1.从html提取信息写入csv

In [ ]:
# 调用外部的py
from demo import *
hello()

# 2.将空缺的日期补齐

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
# First：找到没有进行创作的日子
# 1.读取文件 第一行为列名称
df=pd.read_csv('../csv绘图/info.csv',header=0,sep=',') #filename可以直接从盘符开始，标明每一级的文件夹直到csv文件，header=None表示头部为空，sep=' '表示数据间使用空格作为分隔符，如果分隔符是逗号，只需换成 ‘，’即可。
# 2.按日期进行升序排列
df2 = df.sort_values(by='day')
# 3.提取出 day列  将其转化为 yyyy/mm/dd的格式  将 str=2017/1/27 23:06  转化为  datetime类型
d1 = [datetime.strptime(x, '%Y/%m/%d %H:%M') for x in list(df2["day"])]
#       将 datetime 转化为 str 2017/01/27
d_2017 = [datetime.strftime(x, '%Y/%m/%d') for x in list(d1)]
# 4.创建全年的日子
all_2017=[datetime.strftime(x,'%Y/%m/%d') for x in list(pd.date_range(start='2017/01/01',  end="2017/12/31"))]
# 5.将上两项做差求出没有记录的日子
absent_2017 = [i for i in all_2017 if i not in d_2017]

# 6.将没有记录的日子进行补充  [day:当天,title:空,longitude,latitude,num:0]
dataframe = pd.DataFrame({'day': absent_2017,
                            'title': [None]*len(absent_2017),
                            'longitude': [None]*len(absent_2017),
                            'latitude': [None]*len(absent_2017),
                            'num': [0]*len(absent_2017)})
dataframe.to_csv("./info.csv",index=False, sep=',',mode='a', header=False)


In [ ]:
absent_2017 = ['2017/07/10','2017/07/12','2017/07/11']
absent_2017_np = [time.mktime(time.strptime(x, '%Y/%m/%d')) for x in absent_2017]

In [ ]:
from datetime import datetime
import time       
timeArray = time.strptime('2017/7/11 00:00', '%Y/%m/%d %H:%M')
#转换成时间戳
time.mktime(timeArray)     

In [ ]:
absent_2017_np

# 3.处理重复的日子

## 3.1 手动修改日期 并检查查看

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# First：找到没有进行创作的日子
# 1.读取文件 第一行为列名称
res_path = './data/我的抗战2.0_2020_info.csv'
# res_path = './data/2020_info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 手动修改 日期  
# 截取day的date————2017/02/02  是否唯一进行检查
M = [x.split(" ")[0] for x in df["day"].tolist()]
# 给原 df加一列 date
df['day_date'] = M
# 得到date重复的行  保留出现第一次的值  duplicated(keep=False)
day_date = df[df['day_date'].duplicated(keep=False)]
M = day_date[['day','title','day_date']]
print(len(M))
M

## 3.2自动整理日期
2019/08/19 12:00,,,,0,1566187200.0
2019/08/20 15:02,3.196  转文全部被拒  心情失落  又开始考虑复原,0,0,28,1566284520.0
2019/08/20 15:04,3.197  第一天便无法做到自律 

In [ ]:
def edit_info(res_path):
    # 手动修改 日期  
    # 截取day的date————2017/02/02 12:26  -->  2017/02/02  是否唯一进行检查
    df=pd.read_csv(res_path,header=0,sep=',') 
    M = [x.split(" ")[0] for x in df["day"].tolist()]
    # 给原 df加一列 date
    df['day_date'] = M
    # 得到date重复的行  保留出现第一次的值  duplicated(keep=False)
    day_date = df[df['day_date'].duplicated(keep=False)]
    M = day_date[['day','title','day_date']]
    print("重复的天数为：",len(M))
   
    all_index = M.index.tolist() 
      # 剔除 0 元素
    if 0 in all_index:
        all_index.remove(0)
    # 2.判断 index-1 是否为系统自动生成的 (num值是否为0)
    del_index = []

    for x in all_index:
        if ((df[x:x+1].day_date == '2020/02/13').bool()):
            print("2020/02/13")
        if ((df[x-1:x].num == 0).bool()):
            print("删除行",df[x-1:x].day)
            del_index.append(x-1)
            # df = df.drop([x-1])
        # 3.是——删掉 index-1这一行 
        # 4.修改 df[index] 的 date-1
            temp = df[x:x+1]
            dt = datetime.datetime.strptime(temp.at[temp.index.values[0],'day'],'%Y/%m/%d %H:%M')
            out_date = (dt + datetime.timedelta(days=-1)).strftime('%Y/%m/%d %H:%M')
            temp.day[x] = out_date
    # 5.更新csv文件
    df = df.drop(del_index)
    df.to_csv(res_path,index=False, sep=',')

## 3.3将文件按年进行分表

In [ ]:
# 1.读取文件
res_path = '../info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 2.按照day_date(2018/01/01)  列的前四位  2018进行分表
# day_date_list = list(df['day_date'])
day_date_list = [x.split('/')[0] for x in list(df['day_date'])]
year_list = list(set(day_date_list))
year_list = [eval(x) for x in list(set(day_date_list))]
# 3.将年份由小到大进行排序
year_list.sort()
# 4.分割年份  年份第一次出现的位置进行
all_years = [df[day_date_list.index(str(year_list[x])):day_date_list.index(str(year_list[x]+1))] for x in range(len(year_list)-1)]
# 最后一年
all_years.append(df[day_date_list.index(str(year_list[-1])):])
# 3.输出表
[df_year.to_csv("./data/"+str(year)+"_info.csv",index=False, sep=',')  for (year,df_year) in zip(year_list,all_years)]

## 3.4将没有坐标的日子进行临近填充

In [ ]:
# info_path = 'D:\MyJava\mylifeImg\others\echart\data\demo.csv'
# update_gps_info(info_path)
dir = 'D:\MyJava\mylifeImg\others\echart\data\\'
update_gps_dir(dir)

In [ ]:
# 检验填充后的数据是否还有 gps 为 0 或 Nan
import pandas as pd
import time
info_path = '../info.csv'
m=pd.read_csv(info_path,header=0,sep=',') 
day_date = m[(m['longitude'].isna()) | (m['longitude']=='0')]['day_date']
for x in day_date:
    if time.strptime(x, '%Y/%m/%d')<time.localtime():
        print(x)

# 4.地图视角

## 4.1 历史gps的热力图
var heatmapData = [{
	"lng": 116.191031,
	"lat": 39.988585,
	"count": 10
}, {
	"lng": 116.389275,
	"lat": 39.925818,
	"count": 11
}]

In [ ]:
# 1.制作生成形如上述的mapData.js文件，引入到html中即可
# 2.读取 csv文件的经纬度信息
import pandas as pd
# res_path = './data/2017_info.csv'
res_path = '../info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 3.对经纬度进行 计数统计
# df = df.tail(10)
m = df.groupby(['longitude','latitude']).count()

# 4.输出到js文件
# "lng": 116.191031, "lat": 39.988585, "count": 10
# 将 index 设置为列
m = m.reset_index()
df = pd.DataFrame(columns = ['lng','lat','count'])
df[['lng','lat','count']] = m[['longitude','latitude','day']]
heatmapData = "var heatmapData_local = " + df.to_json(orient = 'records')
# print(heatmapData)
# 打开一个文件
fo = open("./data/heatmapData_local.js", "w")
fo.write(heatmapData)
# 关闭打开的文件
fo.close()

## 4.2把日记标注在地图上

In [115]:
# 1.制作生成形如上述的mapData.js文件，引入到html中即可
# 2.读取 csv文件的经纬度信息
import pandas as pd
# res_path = './data/2017_info.csv'
res_path = '../info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 3.将需要在地图上显示信息 day title longitude latitude输出到js文件

d = pd.DataFrame(columns = ['lng','lat','day','title'])
d[['lng','lat','day','title']] = df[['longitude','latitude','day','title']]
# print(heatmapData)
# 打开一个文件
with open('./data/diarymapData.js', 'w', encoding='utf-8') as file:
    diarymapData = "var diarymapData = " + d.to_json(orient = 'records',force_ascii=False)
   
    file.write(diarymapData)
    # 关闭打开的文件
    file.close()

In [107]:
d = {'name': '张三', 'age': '1'}
print(d) 
jd = d.to_json()
jd

{'name': '张三', 'age': '1'}


AttributeError: 'dict' object has no attribute 'to_json'

In [100]:
# day,title,longitude,latitude
df[['day','title']]

,day,title
0,2017/01/01 12:00,NaN
1,2017/01/02 12:00,NaN
2,2017/01/03 12:00,NaN
3,2017/01/04 12:00,NaN
4,2017/01/05 12:00,NaN
...,...,...
1881,2021/12/27 12:00,NaN
1882,2021/12/28 12:00,NaN
1883,2021/12/29 12:00,NaN
1884,2021/12/30 12:00,NaN
